# RAG Pipeline

1. gpt4all embeddings
2. llama2; 7b params; 6 bits quanitization

# VectorDB Setup

In [1]:
# !pip3 install --upgrade --quiet  langchain langchain-community langchainhub gpt4all chromadb 
# !pip3 install llama-cpp-python

/bin/bash: pip3: command not found


In [1]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
# data = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# all_splits = text_splitter.split_documents(data)

/Users/rohitmalhotra/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [1]:
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.embeddings import HuggingFaceEmbeddings



# embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
model_name = "Salesforce/SFR-Embedding-Mistral"
model_kwargs = {"device": "cuda"}
embedding_function = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)


# embedding_function = SentenceTransformerEmbeddings(model_name="Salesforce/SFR-Embedding-Mistral")


# vectorstore = Chroma(persist_directory="gpt4allDB", embedding_function=GPT4AllEmbeddings())
# vectorstore = Chroma(persist_directory="all-MiniLM-L6-v2DB", embedding_function=embedding_function)
vectorstore = Chroma(persist_directory="sfr-mistralDB/", embedding_function=embedding_function)



/home/trevea/miniconda3/envs/nlp-rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [01:10<00:00, 23.55s/it]


In [2]:
# VectorDB Search Test

question = "What is buggy?"
docs = vectorstore.similarity_search(question)
len(docs)

4

# Chat Agent Setup

In [3]:
!pip install --upgrade --quiet  llama-cpp-python
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q6_K.gguf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.21.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement six~=1.15.0, but you'll have six 1.14.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement typing-extensions~=3.7.4, but you'll have typing-extensions 4.7.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement wrapt~=1.12.1, but you'll have wrapt 1.11.2 which is incompatible.
--2024-03-13 15:58:09--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q6_K.gguf
Resolving huggingface.co (huggingface.co)... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


3.163.189.37, 3.163.189.74, 3.163.189.90, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b0/ca/b0cae82fd4b3a362cab01d17953c45edac67d1c2dfb9fbb9e69c80c32dc2012e/cedd825c615575bcd58eb97cae65ec972afd17b00129f361b6bc3f3633dcd01d?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.Q6_K.gguf%3B+filename%3D%22llama-2-7b-chat.Q6_K.gguf%22%3B&Expires=1710626290&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMDYyNjI5MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iMC9jYS9iMGNhZTgyZmQ0YjNhMzYyY2FiMDFkMTc5NTNjNDVlZGFjNjdkMWMyZGZiOWZiYjllNjljODBjMzJkYzIwMTJlL2NlZGQ4MjVjNjE1NTc1YmNkNThlYjk3Y2FlNjVlYzk3MmFmZDE3YjAwMTI5ZjM2MWI2YmMzZjM2MzNkY2QwMWQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=Cjp0r3QftCon1IN6sVNj9OEedAStEut0iNgX35QnttRNxMmNlQpHdAchiKZiydeDbeyaK2A

In [4]:
from langchain_community.llms import LlamaCpp


n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q6_K.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=True,
)


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q6_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32

# RAG

In [5]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnablePick
from langchain_core.prompts.chat import HumanMessagePromptTemplate, PromptTemplate

rag_prompt = hub.pull("rlm/rag-prompt")
# rag_prompt.messages
prompt = HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use one sentence maximum and keep the answer CONCISE. Keep the answer CONCISE.\nQuestion: {question} \nContext: {context} \nAnswer:"))
rag_prompt.messages = [prompt]

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


retriever = vectorstore.as_retriever()
qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)



In [7]:
f = open("SubmissionData/test/questions.txt", "r")
questions = f.readlines()
f.close()

questions = [i.strip() for i in questions]

In [ ]:
from tqdm import tqdm

answers = []
for i in tqdm(range(len(questions))):
  response = qa_chain.invoke(questions[i])
  answers.append(response)

  0%|          | 0/79 [00:00<?, ?it/s]
llama_print_timings:        load time =    7679.45 ms
llama_print_timings:      sample time =      12.89 ms /    79 runs   (    0.16 ms per token,  6128.78 tokens per second)
llama_print_timings: prompt eval time =    7679.10 ms /   504 tokens (   15.24 ms per token,    65.63 tokens per second)
llama_print_timings:        eval time =   13675.96 ms /    78 runs   (  175.33 ms per token,     5.70 tokens per second)
llama_print_timings:       total time =   21570.50 ms /   582 tokens
  1%|▏         | 1/79 [00:22<28:44, 22.11s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =    7679.45 ms
llama_print_timings:      sample time =       9.91 ms /    63 runs   (    0.16 ms per token,  6357.86 tokens per second)
llama_print_timings: prompt eval time =    5915.81 ms /   371 tokens (   15.95 ms per token,    62.71 tokens per second)
llama_print_timings:        eval time =   10732.68 ms /    62 runs   (  173.11 ms per token,     5.

In [ ]:
def writeListToFile(filename, l):
    f = open(filename, "w")
    for i in l:
        f.write(i + "\n")
    f.close()



writeListToFile("SubmissionData/system_outputs/system_output_3.txt", answers)